# Lab 7 - GAN Implementation
## Name: Sarim Aeyzaz
## Roll No: i21-0328


In [ ]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import torch.backends.cudnn as cudnn
import torchvision.datasets as dset
import torch.optim as optim
import torch.nn as nn
import numpy as np
import torch

CUDA = True     # Change to False for CPU training
DATA_PATH = '~/Data/mnist'
BATCH_SIZE = 2048
IMAGE_CHANNEL = 1
X_DIM, Z_DIM = 64, 100
G_HIDDEN, D_HIDDEN = 64, 64
REAL_LABEL, FAKE_LABEL = 1, 0
EPOCH_NUM = 12
FAKE_LABEL = 0
lr = 2e-4
seed = 1            # Change to None to get different results at each run

CUDA = CUDA and torch.cuda.is_available()
print(f"PyTorch version: {torch.__version__}")
if CUDA:
    print(f"CUDA version: {torch.version.cuda}\n")

if seed is None:
    seed = np.random.randint(1, 10000)
print("Random Seed: ", seed)
np.random.seed(seed)
torch.manual_seed(seed)

if CUDA:
    torch.cuda.manual_seed(seed)
cudnn.benchmark = True      # May train faster but cost more memory

PyTorch version: 2.1.0+cu118
CUDA version: 11.8

Random Seed:  1


In [ ]:
dataset = dset.MNIST(root=DATA_PATH, download=True,
                      transform=transforms.Compose([
                      transforms.Resize(X_DIM),
                      transforms.ToTensor(),
                      transforms.Normalize((0.5,), (0.5,))
                      ]))

assert dataset
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
device = torch.device("cuda:0" if CUDA else "cpu")

In [ ]:
def weights_init(m):
    """custom weights initialization
    """
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        m.weight.data.normal_(0.0, 0.02)
    elif classname.find('BatchNorm') != -1:
        m.weight.data.normal_(1.0, 0.02)
        m.bias.data.fill_(0)


class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            # 1st layer
            nn.ConvTranspose2d(Z_DIM, G_HIDDEN * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 8),
            nn.ReLU(True),
            # 2nd layer
            nn.ConvTranspose2d(G_HIDDEN * 8, G_HIDDEN * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 4),
            nn.ReLU(True),
            # 3rd layer
            nn.ConvTranspose2d(G_HIDDEN * 4, G_HIDDEN * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN * 2),
            nn.ReLU(True),
            # 4th layer
            nn.ConvTranspose2d(G_HIDDEN * 2, G_HIDDEN, 4, 2, 1, bias=False),
            nn.BatchNorm2d(G_HIDDEN),
            nn.ReLU(True),
            # output layer
            nn.ConvTranspose2d(G_HIDDEN, IMAGE_CHANNEL, 4, 2, 1, bias=False),
            nn.Tanh()
        )

    def forward(self, input):
        return self.main(input)


class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        self.main = nn.Sequential(
            # 1st layer
            nn.Conv2d(IMAGE_CHANNEL, D_HIDDEN, 4, 2, 1, bias=False),
            nn.LeakyReLU(0.2, inplace=True),
            # 2nd layer
            nn.Conv2d(D_HIDDEN, D_HIDDEN * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 2),
            nn.LeakyReLU(0.2, inplace=True),
            # 3rd layer
            nn.Conv2d(D_HIDDEN * 2, D_HIDDEN * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 4),
            nn.LeakyReLU(0.2, inplace=True),
            # 4th layer
            nn.Conv2d(D_HIDDEN * 4, D_HIDDEN * 8, 4, 2, 1, bias=False),
            nn.BatchNorm2d(D_HIDDEN * 8),
            nn.LeakyReLU(0.2, inplace=True),
            # output layer
            nn.Conv2d(D_HIDDEN * 8, 1, 4, 1, 0, bias=False),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input).view(-1, 1).squeeze(1)

In [ ]:
def train_discriminator(optimizerD, x_real, real_label):
      optimizerD.zero_grad()

      # Update D with real data
      read_predictions = netD(x_real).to(device)
      real_loss = criterion(read_predictions, real_label)

      # Fake Images
      z_noise = torch.randn(x_real.size(0), Z_DIM, 1, 1, device=device)
      fake_images = netG(z_noise).to(device)

      # Update D with fake data
      fake_predictions = netD(fake_images).to(device)
      fake_loss = criterion(fake_predictions, fake_label)

      loss = real_loss + fake_loss
      loss.backward()
      optimizerD.step()

      return real_loss.item(), fake_loss.item()

In [ ]:
def train_generator(optimizerG, real_label):
    optimizerG.zero_grad()

    # Fake Images
    z_noise = torch.randn(x_real.size(0), Z_DIM, 1, 1, device=device)
    fake_images = netG(z_noise).to(device)

    # Update G with fake data
    fake_predictions = netD(fake_images).to(device)
    loss = criterion(fake_predictions, real_label)
    loss.backward()
    optimizerG.step()

    return loss.item()

In [ ]:
netG = Generator().to(device)
netG.apply(weights_init)
print(netG)

netD = Discriminator().to(device)
netD.apply(weights_init)
print(netD)

criterion = nn.BCELoss()
optimizerD = optim.Adam(netD.parameters(), lr=lr, betas=(0.5, 0.999))
optimizerG = optim.Adam(netG.parameters(), lr=lr, betas=(0.5, 0.999))

Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 1, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)


In [ ]:
# Note: I've increased batch size from 512 to 2048 in order make it run a bit faster on my system

for epoch in range(EPOCH_NUM):
    for i, data in enumerate(dataloader):
        x_real = data[0].to(device)

        real_label = torch.full((x_real.size(0),), REAL_LABEL, device=device, dtype=torch.float32)
        fake_label = torch.full((x_real.size(0),), FAKE_LABEL, device=device, dtype=torch.float32)

        real, fake = train_discriminator(optimizerD, x_real, real_label)
        loss = train_generator(optimizerG, real_label)

        print(f'Epoch {epoch} [{i}/{len(dataloader)}] Loss D (Real): {real: .4f} Loss D (Fake): {fake: .4f} Loss G: {loss: .4f}')

Epoch 0 [0/30] Loss D (Real):  1.1149 Loss D (Fake):  1.0050 Loss G:  2.4114
Epoch 0 [1/30] Loss D (Real):  0.0022 Loss D (Fake):  3.0609 Loss G:  3.3350
Epoch 0 [2/30] Loss D (Real):  0.0049 Loss D (Fake):  1.7216 Loss G:  5.7054
Epoch 0 [3/30] Loss D (Real):  0.0553 Loss D (Fake):  0.5069 Loss G:  6.7337
Epoch 0 [4/30] Loss D (Real):  0.1690 Loss D (Fake):  0.2512 Loss G:  6.2437
Epoch 0 [5/30] Loss D (Real):  0.1993 Loss D (Fake):  0.3869 Loss G:  6.7481
Epoch 0 [6/30] Loss D (Real):  0.1561 Loss D (Fake):  0.2109 Loss G:  7.2319
Epoch 0 [7/30] Loss D (Real):  0.0834 Loss D (Fake):  0.1574 Loss G:  7.3065
Epoch 0 [8/30] Loss D (Real):  0.0478 Loss D (Fake):  0.1302 Loss G:  7.4557
Epoch 0 [9/30] Loss D (Real):  0.0473 Loss D (Fake):  0.1278 Loss G:  7.7582
Epoch 0 [10/30] Loss D (Real):  0.0543 Loss D (Fake):  0.1178 Loss G:  8.1559
Epoch 0 [11/30] Loss D (Real):  0.0560 Loss D (Fake):  0.0960 Loss G:  8.0973
Epoch 0 [12/30] Loss D (Real):  0.0549 Loss D (Fake):  0.1337 Loss G:  9.1